In [1]:
!pip install torch
!pip install transformers
!pip install accelerate
!pip install sentence_transformers
!pip install einops
!pip install faiss-cpu
!pip install langchain
!pip install langchain-community
!pip install unstructured

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import TextIteratorStreamer
from threading import Thread

In [3]:
# Prompt template
template = """
=======
{context}
=======
Question: {question}
Output:\n"""

QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

In [4]:
model = "C:\\Users\\acer alan\Desktop\\Phi-3-mini-128k"
model = AutoModelForCausalLM.from_pretrained(
    "C:\\Users\\acer alan\Desktop\\Phi-3-mini-128k",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
#tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\acer alan\Desktop\\Phi-3-mini-128k")
# sentence transformers to be used in vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-base-v4",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Returns a faiss vector store retriever given a txt file
def prepare_vector_store_retriever(filename):
    # Load data
    loader = UnstructuredFileLoader(filename)
    raw_documents = loader.load()

    # Split the text
    text_splitter = CharacterTextSplitter(
        separator="\n\n", chunk_size=800, chunk_overlap=0, length_function=len
    )

    documents = text_splitter.split_documents(raw_documents)

    # Creating a vectorstore
    vectorstore = FAISS.from_documents(
        documents, embeddings, distance_strategy=DistanceStrategy.DOT_PRODUCT
    )

    return VectorStoreRetriever(vectorstore=vectorstore, search_kwargs={"k": 2})

In [6]:
# Retrieveal QA chian
def get_retrieval_qa_chain(text_file, hf_model):
    retriever = default_retriever
    if text_file != default_text_file:
        retriever = prepare_vector_store_retriever(text_file)

    chain = RetrievalQA.from_chain_type(
        llm=hf_model,
        retriever=retriever,
        chain_type_kwargs={"prompt": QA_PROMPT},
    )
    return chain

In [7]:
# Generates response using the question answering chain defined earlier
#def generate(question, answer, text_file, max_new_tokens):
import time
import json
def generate(question, text_file, max_new_tokens):
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=300.0
    )
    phi3_pipeline = pipeline(
        "text-generation",
        tokenizer=tokenizer,
        model=model,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        device_map="auto",
        streamer=streamer,
    )

    hf_model = HuggingFacePipeline(pipeline=phi3_pipeline)
    qa_chain = get_retrieval_qa_chain(text_file, hf_model)

    query = f"{question}"

    if len(tokenizer.tokenize(query)) >= 512:
        query = "Repeat 'Your question is too long!'"

    thread = Thread(target=qa_chain.invoke, kwargs={"input": {"query": query}})
    thread.start()

    ''''
    response = ""
    for token in streamer:
        response += token

    return response.strip()
    '''
    response = ''
    for token in streamer:
        response += token

        # Print each character with a delay
        for char in token:
            print(char, end='', flush=True)
            time.sleep(0.1)  # Adjust the delay as needed
        print("", end='')

    with open('data.json','w') as file:
        json.dump(response.strip(),file)
    #return response.strip()
    


In [8]:
default_text_file = "C:\\Users\\acer alan\\Desktop\\Tears_of_Queen.txt"
default_retriever = prepare_vector_store_retriever(default_text_file)

In [9]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['bye','quit','exit']:
        print("Chatbot: Goodbye")
        break
    generate(user_input,"C:\\Users\\acer alan\\Desktop\\Tears_of_Queen.txt",500)

The model 'Phi3ForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus


- Response: The capital of China is Beijing.

The model 'Phi3ForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus


- response: Queen of Tears is a South Korean television series.

The model 'Phi3ForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus


- Response: The capital of Taiwan is Taipei. Taiwan, officially known as the Republic of China (ROC), is an island in East Asia located in the northwest Pacific Ocean. Taipei, the largest city in Taiwan, serves as the political, economic, cultural, and educational center of the island. It is where the President's official residence, the Presidential Office Building, is located, as well as the Legislative Yuan, the Executive Yuan, and the Judicial Yuan, which are the three branches of the government of the Republic of China.

Taipei is also known for its vibrant street food scene, modern architecture, and historical landmarks. The city is home to the iconic Taipei 101, which was the world's tallest building from its completion in 2004 until the completion of the Burj Khalifa in Dubai in 2010. Although Taipei 101 has since been surpassed in height, it remains a symbol of Taiwan's economic growth and architectural prowess.

In addition to its political significance, Taipei is a hub for t